# Fichier de test de récupération des horaires de bus tcl

In [1]:
# Librairies
import json
import pandas as pd
import requests
import io
from datetime import datetime
import math
import serial
import serial.tools.list_ports
import time

## Place arrets infos

[Lien pour voir les ids des arrêts](https://data.grandlyon.com/jeux-de-donnees/points-arret-reseau-transports-commun-lyonnais/donnees)

| id | nom | desserte | gid |
| --- | --- | --- | --- |
| 11002 | Monchat Place Ronde | C9A:R | 686 |
| 43120 | Monchat Place Ronde | C16A:R | 687 |
| 43119 | Monchat Place Ronde | C26A:A | 688 |
| 42616 | Monchat Place Ronde | C9A:A | 685 |
| 47226 | Aubepins | 25A:A,C26A:R | 9266 |
| 85 | Aubepins | 25A:A,C26A:A |850 |

## Arduino

In [2]:
import serial
import serial.tools.list_ports
import time

In [3]:
def connection_arduino_choix():
    print("Recherche d'un port serie...")

    ports = serial.tools.list_ports.comports(include_links=False)

    if (len(ports) != 0): # on a trouvé au moins un port actif

        if (len(ports) > 1):     # affichage du nombre de ports trouvés
            print (str(len(ports)) + " ports actifs ont ete trouves:") 
        else:
            print ("1 port actif a ete trouve:")

        ligne = 1

        for port in ports :  # affichage du nom de chaque port
            print(str(ligne) + ' : ' + port.device)
            ligne = ligne + 1

        portChoisi = input('Ecrivez le numero du port desire:')

        print('1: 9600   2: 38400    3: 115200')

        baud = input('Ecrivez le baud rate desire:')

        if (baud == 1):
            baud = 9600
        if (baud == 2):
            baud = 38400
        if (baud == 3):
            baud = 115200

        # on établit la communication série
        device_selected=ports[int(portChoisi) - 1].device
        arduino = serial.Serial(device_selected, baud, timeout=1)
        print('Connexion a ' + arduino.name + ' a un baud rate de ' + str(baud))

        # si on reçoit un message, on l'affiche
        while True:
            data = arduino.readline()[:-2]
            if data:
                print(data)

    else: # on n'a pas trouvé de port actif
        print("Aucun port actif n'a ete trouve")

In [4]:
def envoyer_message_arduino(port, baud, message = 'Hello, Arduino!'):
    # Remplacez 'COM3' par le nom du port série utilisé par votre carte Arduino
    arduino = serial.Serial(port, baud)

    # Attendre que la connexion soit établie
    time.sleep(2)

    # Envoyer des données à l'Arduino
    arduino.write(message.encode('ascii'))

    # Fermer la connexion
    arduino.close()

# Autre test *RESET*

#### [Page web](https://rdata-grandlyon.readthedocs.io/fr/latest/services.html?highlight=temps%20r%C3%A9el#les-services-offerts-par-grandlyon-data) 



Il est également possible de filtrer les objets renvoyés selon une valeur d’attribut avec une url de la forme :

https://download.data.grandlyon.com/ws/<service>/<table_schema>.<table_name>/all.json?field=<attribut>&value=<valeur>

Exemple : https://download.data.grandlyon.com/ws/grandlyon/abr_arbres_alignement.abrarbre/all.json?field=essencefrancais&value=Marronnier%20de%20Virginie

all.json contient aussi des informations supplémentaires liées à la pagination, à savoir des liens vers les pages précédentes et suivantes sous la forme d’une URL reprenant la valeur de maxfeatures utilisée pour la page en cours et modifiant la valeur du paramètre « start » en fonction de la page en cours.

Exemple : https://download.data.grandlyon.com/ws/grandlyon/gip_proprete.gipdecheterie/all.json?maxfeatures=5&start=10

Cette URL retourne les enregistrements 10 à 15 de la couche déchetterie.

Les services REST-JSON sont ainsi particulièrement adaptés à la constitution de listes de valeurs, de tableaux et de grilles paginés, d’interface de navigation au sein des données.

Ensuite, à la mode Django, paramètres personnalisés sous la forme champs__opérateur, avec champ = nom du champ cible et opérateur à choisir parmi eq, gt, gte, lt, lte, in

Exemple : https://download.recette.data.grandlyon.com/ws/grandlyon/abr_arbres_alignement.abrarbre/all.json?codeinsee__eq=69116&commune__in=LIMONEST,BRON&dateplantation__gte=2009-03-01&gid__in=6795,6798

Les opérateurs: * eq : (equal) égalité * gt : (greater than) strictement plus grand que * gte : (greater than or equal) plus grand ou égal * lt : (lesser than) strictement plus petit que * lte : (lesser than or equal) plus petit ou égal * in : (in) dans la liste, les éléments de la liste sont séparés par des virgules.

Services REST (en CSV)
Exemple : https://download.data.grandlyon.com/ws/grandlyon/gip_proprete.gipdecheterie/all.csv?maxfeatures=5&start=10

De la même façon que l’on requête le service JSON, on peut demander un extrait CSV en remplaçant l’extension « .json » de l’URL par « .csv ».

Il est possible de remplacer le séparateur décimal en ajoutant “ds=,” ou “ds=.” dans la requête.

Le séparateur de colonne peut aussi être changé en utilisant l’option « separator=; » par exemple.

Un paramètre supplémentaire « geometry=on » (off par défaut) ajoute une colonne « WKT » contenant la géométrie de l’objet au format [WKT](https://fr.wikipedia.org/wiki/Well-known_text)

#### explication url

url = https://download.data.grandlyon.com/ws/rdata/tcl_sytral.tclpassagearret/all.json?id__eq=85&ligne__eq=C26A&type__eq=E

base url = https://download.data.grandlyon.com/ws/rdata/tcl_sytral.tclpassagearret/all.json


#### Code

In [5]:
# Librairies
import json
import pandas as pd
import requests
import io
from datetime import datetime
import math
import serial
import serial.tools.list_ports
import time

In [6]:
# Requête
username=""
password=""
base_url = "https://download.data.grandlyon.com/ws/rdata/tcl_sytral.tclpassagearret/all.csv"
auth = (username, password)

# Arduino
port_selected='COM3'
# port_selected_unix='/dev/ttyUSB0'
baud_selected=9600

# Arrêt
id_arret=85  # Aubepin
ligne="C26A" # Ligne C26 direction Grange Blanche

In [7]:
# Fait la requete pour obtenir les données d'un arrêt et d'une ligne
# Renvoie un tab a 2 dim avec 1ere dim = nom du champ et 2eme valeur (un peu dico mais appelé ici DataFrame)
def GetData(auth, id_arret, ligne, type = "E"):
    param = f"id__eq={id_arret}&ligne__eq={ligne}&type__eq=E"
    param_csv="maxfeatures=-1&ds=.&separator=;&start=1"
    url = base_url + "?" + param + "&" + param_csv
    response = requests.get(url, auth=auth)
    text_csv = response.text
    return pd.read_csv(io.StringIO(text_csv), sep=";").sort_values("heurepassage")

print(GetData(auth=auth, id_arret=id_arret, ligne=ligne).to_string())

   id ligne       direction delaipassage type         heurepassage  idtarretdestination                 coursetheorique    gid      last_update_fme
0  85  C26A  Grange Blanche       13 min    E  2023-06-04 17:48:06                 1095  C26A-0607_x0040__x0023_:1:1:26  12566  2023-06-04 17:34:02
1  85  C26A  Grange Blanche       30 min    E  2023-06-04 18:05:00                 1095  C26A-0607_x0040__x0023_:4:1:17  12677  2023-06-04 17:34:02


In [8]:
# Créer message
def creer_message_delaipassage(data):
    liste_msg = []
    lenMaxLigne = 16
    
    for index, passage in data.iterrows():
        if index == 0:
            texte = "Prochain "
        elif index == 1:
            texte = "Suivant "
        else:
            texte = "Apres "

        # Date et heure spécifiée au format "AAAA-MM-JJ HH:MM:SS"
        date_passage_str = passage['heurepassage']
        # Conversion de la chaîne de caractères en objet datetime
        date_passage = datetime.strptime(date_passage_str, "%Y-%m-%d %H:%M:%S")
        # Obtenir l'heure actuelle
        now = datetime.now()
        # Calculer la différence de temps en secondes
        delta_seconds = (date_passage - now).total_seconds()
        # Convertir la différence de temps en minutes et donne un string
        delta_minutes_str = str(math.floor(delta_seconds / 60)) # Arrondi à l'inférieur
        
        space = ' ' * (lenMaxLigne - len(texte) - len(delta_minutes_str) - len(" min"))
        liste_msg.append(texte + space + delta_minutes_str + " min")
    return liste_msg

liste_msg = creer_message_delaipassage(GetData(auth, id_arret,ligne))
for msg in  liste_msg:
    print(msg)

Prochain  13 min
Suivant   30 min


In [9]:
def etablir_connexion_arduino(port, baud, first_message):
    # Remplacez 'COM3' par le nom du port série utilisé par votre carte Arduino
    arduino = serial.Serial(port, baud)
    # Attendre que la connexion soit établie
    time.sleep(2)
    return arduino

# Fermer la connexion
def fermer_connexion(arduino):
    arduino.close()

# Envoie sur l'arduino message sur la ligne X
def envoyer_message_arduino_ligne_x(arduino, message, ligne_x):
    message = str(ligne_x) + ';' + message
    arduino.write(message.encode())
    
# Envoie sur l'arduino le messages un des prochains passages
def envoyer_message_arret_arduino(arduino, message):
    envoyer_message_arduino_ligne_x(arduino, message, 0)


# Envoie sur l'arduino le message d'info sur l'horaire
def envoyer_message_passage_arduino(arduino, message):
    envoyer_message_arduino_ligne_x(arduino, message, 1)

# Steam

pip install python-steam

In [5]:
import requests
import steam.webauth as wa
import steam.webapi as api

# variables
id_jeu_valheim = 892970
# Remplacez "YOUR_API_KEY" par votre clé d'API Steam
api_key = ""

# Remplacez "YOUR_STEAM_ID" par votre ID Steam
steam_id = ""

In [8]:
def recup_username(api_key, steam_id_user):
    username = "Not found"
    # URL de l'API Steam pour obtenir les informations de l'utilisateur
    url_player_info = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={api_key}&steamids={steam_id_user}"
    # Faites une requête GET à l'API Steam pour obtenir les informations de l'utilisateur
    response = requests.get(url_player_info)
    data = response.json()

    if data.get("response"):
        player_info = data["response"]["players"][0]
        player_name = player_info["personaname"]
        username = player_name
    return username


In [9]:
def recup_liste_amis(api_key, steam_id):
    liste_amis = {}
    # URL de l'API Steam pour obtenir la liste d'amis
    url_friends = f"http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?steamid={steam_id}&key={api_key}&relationship=friend"
    # Faites une requête GET à l'API Steam pour obtenir la liste d'amis
    response = requests.get(url_friends)
    data = response.json()

    if data.get("friendslist"):
        friends = data["friendslist"]["friends"]
        for friend in friends:
            # Obtenir l'ID Steam de l'ami
            friend_id = friend["steamid"]
            username = recup_username(api_key, friend_id)
            liste_amis[friend_id] = username
    return liste_amis

print(recup_liste_amis(api_key, steam_id))

{'76561198024743839': 'guillaume2181', '76561198054626463': 'Mda00', '76561198055041213': 'ArchmagosLoki', '76561198066271546': 'Boneshiver', '76561198076051119': 'RoG spoukies', '76561198079109507': 'GrandNago', '76561198089632433': 'hecafriquain', '76561198096377419': 'nicolas.vdt', '76561198108749957': 'Hanarch', '76561198117363527': 'Haxel', '76561198119510047': 'Port Salut', '76561198120953749': 'Lauradach', '76561198132418261': 'Matou21850', '76561198135031045': 'fff60000', '76561198154180013': 'thepoWerdark', '76561198154989286': 'Oriuck', '76561198163118048': '[BM]DARKSILVER', '76561198163400056': 'So La sekte', '76561198169603156': 'ramalac', '76561198269374767': 'Herrlockskun', '76561198297960415': 'ReDodge', '76561198315699250': 'Maubeuge', '76561198407505536': 'remremj', '76561198412543850': 'Nathus', '76561198827388775': 'Matorqui', '76561198829418646': 'Huriiko', '76561198845292735': 'archidamos429'}


In [10]:
def friends_sur_un_jeu(api_key, liste_amis, id_jeu):
    nbAmis = 0
    for id_ami in liste_amis.keys():
        # URL de l'API Steam pour obtenir les infos sur l'ami
        url_info_ami = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={api_key}&steamids={id_ami}"
        # Faites une requête GET à l'API Steam pour obtenir les jeux possédés par l'ami
        response = requests.get(url_info_ami)
        info_ami = response.json()
        if info_ami.get("response"):
            # print(info_ami)
            if info_ami["response"]["players"][0].get("gameid"):
                if info_ami["response"]["players"][0]["gameid"] == str(id_jeu):
                    print(liste_amis[id_ami] + " est sur le jeu " + str(id_jeu))
                    nbAmis +=1
    return nbAmis

print("Copains sur Valheim : " + str(friends_sur_un_jeu(api_key, recup_liste_amis(api_key, steam_id), id_jeu_valheim)))

guillaume2181 est sur le jeu 892970
GrandNago est sur le jeu 892970
Copains sur Valheim : 2


In [14]:
# def creer_message_joueur_sur_valheim():

In [15]:
liste_amis = recup_liste_amis(api_key, steam_id)
print(liste_amis)

{'76561198024743839': 'guillaume2181', '76561198054626463': 'Mda00', '76561198055041213': 'ArchmagosLoki', '76561198066271546': 'Boneshiver', '76561198076051119': 'RoG spoukies', '76561198079109507': 'GrandNago', '76561198089632433': 'hecafriquain', '76561198096377419': 'nicolas.vdt', '76561198108749957': 'Hanarch', '76561198117363527': 'Haxel', '76561198119510047': 'Port Salut', '76561198120953749': 'Lauradach', '76561198132418261': 'Matou21850', '76561198135031045': 'fff60000', '76561198154180013': 'thepoWerdark', '76561198154989286': 'Oriuck', '76561198163118048': '[BM]DARKSILVER', '76561198163400056': 'So La sekte', '76561198169603156': 'ramalac', '76561198269374767': 'Herrlockskun', '76561198297960415': 'ReDodge', '76561198315699250': 'Maubeuge', '76561198407505536': 'remremj', '76561198412543850': 'Nathus', '76561198827388775': 'Matorqui', '76561198829418646': 'Huriiko', '76561198845292735': 'archidamos429'}


# Programme 

In [16]:
def prog(auth_url, id_arret, ligne, port_arduino, baud_arduino):

    time_actu_data = 60 

    arduino = etablir_connexion_arduino(port_arduino, baud_arduino, first_message="Helloworld")
    
    envoyer_message_arret_arduino(arduino, "Arret Aubepins")
    time.sleep(10)

    i=0
    try:
        start_time = time.time()
        while(True):
            current_time = time.time()
            if current_time - start_time >= time_actu_data:
                # Récupère les data
                data = GetData(auth, id_arret, ligne)
                start_time = current_time

            liste_prochain_passage = creer_message_delaipassage(data)
            for passage in liste_prochain_passage:
                envoyer_message_passage_arduino(arduino, passage)
                time.sleep(2)  
    except e:
        fermer_connexion(arduino)      
        print("Erreur" + str(e))

prog(auth, id_arret, ligne, port_selected, baud_selected)

SerialException: could not open port 'COM3': FileNotFoundError(2, 'Le fichier spécifié est introuvable.', None, 2)

# RabbitMQ

In [ ]:
#!/usr/bin/env python
import pika
queue_name="arduino"

## Producer

In [ ]:
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()
# Créer la queue
channel.queue_declare(queue=queue_name)

<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=1', 'queue=arduino'])>"])>

In [ ]:
credentials = pika.PlainCredentials(username="guest", password="guest")
parameters = pika.ConnectionParameters(host="localhost", 
                                        port=5672,
                                        virtual_host="/",
                                        credentials=credentials)
connection = pika.BlockingConnection(parameters)
channel= connection.channel()

In [ ]:
for i in range(0,1):
    channel.basic_publish(exchange='',
                        routing_key=queue_name,
                        body='Hello World! ' + str({i}))
    print(" [x] Sent 'Hello World! " + str({i}))

 [x] Sent 'Hello World! {0}


In [ ]:
connection.close()

## Consumer

In [ ]:
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()
channel.queue_declare(queue='hello')

<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=4', 'queue=hello'])>"])>

In [ ]:
def callback(ch, method, properties, body):
    print(" [x] Received %r" % body)

In [ ]:
channel.basic_consume(queue='hello',
                      auto_ack=True,
                      on_message_callback=callback)

'ctag1.b73c07347ca34a6a8317ee753b0c14c7'

In [ ]:
print(' [*] Waiting for messages. To exit press CTRL+C')
channel.start_consuming()

 [*] Waiting for messages. To exit press CTRL+C
 [x] Received b'Hello World!'
 [x] Received b'Hello World!'
 [x] Received b'Hello World!'
 [x] Received b'Hello World!'


KeyboardInterrupt: 

## Receveur : delapineur

```
pip install pika
pip install sys
pip install os
pip install serial.tools.list_ports
pip install time
pip install datetime
```

In [ ]:
#!/usr/bin/env python
import pika, sys, os
import serial.tools.list_ports
import time
from datetime import datetime

# Arduino
arduino = None
port_arduino ='COM3'
# port_arduino  = '/dev/ttyACM0'
baud = 9600
MSG_VIDE_ARDUINO="                ;                "

## Queue
# Queue parametre
ip_rabbitMQ_service="localhost"
port_rabbitMQ_service=5672
virtual_host="/"
queue_name=""
username=""
password=""

# Queue : format message 
MSG_TEST="Toto;2023-06-04 00:35:45;2024-06-04 00:35:45;5;Message ligne 1;Message ligne2"

FORMAT_MSG="nom_publisher;date_envoie(%Y-%m-%d %H:%M:%S);date_peremption(%Y-%m-%d %H:%M:%S);duree_affichage(secondes);ligne1_msg;ligne2_msg"
NOM_PUBLISHER=0
DATE_ENVOIE=1
DATE_PEREMPTION=2
DUREE_AFFICHAGE=3
LIGNE1_MSG=4
LIGNE2_MSG=5

# Format du message : 
# nom_publisher
# date_envoie => %Y-%m-%d %H:%M:%S
# date_peremption => %Y-%m-%d %H:%M:%S.%3N"
# duree_affichage => en seconde
# ligne1_msg
# ligne2_msg

def callback(ch, method, properties, body):
    # print(" [x] Received %r" % body)
    # print(" [x] Received %r" % body + str(body.type))
    try:
        params_msg = check_msg_ok(str(body))
        if(check_validite_msg(params_msg)):
                message_complet = params_msg[LIGNE1_MSG] + ";" + params_msg[LIGNE2_MSG]
                arduino.write(message_complet.encode())
                print("Affichage pendant " + str(params_msg[DUREE_AFFICHAGE]) + "s du message : " + message_complet)
                time.sleep(params_msg[DUREE_AFFICHAGE])
                arduino.write(MSG_VIDE_ARDUINO.encode())
    except ValueError as e:
        print("Erreur format : " + str(e))
    except Exception as e:
        print("Erreur : " + str(e))

    
def split_msg(data: str, separator : str = ";") -> list:
    return data.split(separator)

def is_ascii(s):
    return all(ord(c) < 128 for c in s)

# Renvoie la liste des paramètres correctement convertie pour un traitement
def check_msg_ok(message ) -> list:
    params_msg = split_msg(message)
    
    # Erreur de format pour le message
    if(len(params_msg) != 6):
        raise ValueError("Format message pas du bon type : " + ";".join(params_msg) + " et format attendu : " + FORMAT_MSG)

    # Vérifiacation des chaines ascii
    if(is_ascii(params_msg[NOM_PUBLISHER]) == False):
        raise ValueError("Erreur valeur non ASCII pour le nom du publisher : "+ params_msg[NOM_PUBLISHER])
    if(is_ascii(params_msg[LIGNE1_MSG]) == False):
        raise ValueError("Erreur valeur non ASCII pour le msg sur la ligne 1 : "+ params_msg[LIGNE1_MSG])
    if(is_ascii(params_msg[LIGNE2_MSG]) == False):
        raise ValueError("Erreur valeur non ASCII pour le msg sur la ligne 1 : "+ params_msg[LIGNE2_MSG])

    # Récupération date
    try: # Récupération de la date d'envoie
        date_envoie = datetime.strptime(params_msg[DATE_ENVOIE], "%Y-%m-%d %H:%M:%S")
        params_msg[DATE_ENVOIE] = date_envoie
    except ValueError as e:
       print("Erreur date d'envoie : " + str(e))
    try: # Récupération de la date de peremption
        date_peremption = datetime.strptime(params_msg[DATE_PEREMPTION], "%Y-%m-%d %H:%M:%S")
        params_msg[DATE_PEREMPTION] = date_peremption
    except ValueError as e:
       print("Erreur date de péremption :" + str(e))

    try: # Récupération duree affichage
        duree_affichage = int(params_msg[DUREE_AFFICHAGE])
        params_msg[DUREE_AFFICHAGE] = duree_affichage
    except ValueError as e:
        print("Erreur format duree affichage :" + str(e))

    return params_msg

# Check si la date de peremption n'est pas dépassee
def check_validite_msg(params_msg: list) -> bool:
    # if(time.time() + params_msg[DUREE_AFFICHAGE] > params_msg[DATE_PEREMPTION]):
    if(datetime.now() > params_msg[DATE_PEREMPTION]):
        return False
    else:
        return True

def main():

    credentials = pika.PlainCredentials(username=username, password=password)
    parameters = pika.ConnectionParameters(host=ip_rabbitMQ_service, 
                                           port=port_rabbitMQ_service,
                                           virtual_host=virtual_host,
                                           credentials=credentials)
    connection = pika.BlockingConnection(parameters)
    channel = connection.channel()
    # channel.queue_declare(queue=queue_name)

    channel.basic_consume(queue=queue_name, on_message_callback=callback, auto_ack=True)

    print(' [*] Waiting for messages. To exit press CTRL+C')
    channel.start_consuming()

if __name__ == '__main__':
    try:
        # Remplacez 'COM3' par le nom du port série utilisé par votre carte Arduino
        arduino = serial.Serial(port_arduino, baud)
        # Attendre que la connexion soit établie
        time.sleep(2)

        main()
    except KeyboardInterrupt:
        print('Interrupted')
        try:
            sys.exit(0)
        except SystemExit:
            os._exit(0)
    finally:
        arduino.close()
        connection.close()

AttributeError: 'NoneType' object has no attribute 'close'

## Emetteur Lapin TCL

In [1]:
#!/usr/bin/env python
# Librairies
import pika, sys, os
import sys
import pandas as pd
import requests
import io
from datetime import datetime, timedelta
import math
import time

# Format du message : 
# nom_publisher
# date_envoie => %Y-%m-%d %H:%M:%S
# date_peremption => %Y-%m-%d %H:%M:%S.%3N"
# duree_affichage => en seconde
# ligne1_msg
# ligne2_msg
# Queue : format message 
MSG_TEST="Toto;2023-06-04 00:35:45;2024-06-04 00:35:45;5;Message ligne 1;Message ligne2"
FORMAT_MSG="nom_publisher;date_envoie(%Y-%m-%d %H:%M:%S);date_peremption(%Y-%m-%d %H:%M:%S);duree_affichage(secondes);ligne1_msg;ligne2_msg"

## Queue
# Queue parametre
ip_rabbitMQ_service="localhost"
port_rabbitMQ_service=5672
virtual_host="/"
queue_name=""
usernameQueue=""
passwordQueue=""

## Param TCL
usernameTCL=""
passwordTCL=""
base_url = "https://download.data.grandlyon.com/ws/rdata/tcl_sytral.tclpassagearret/all.csv"
authTCL = (usernameTCL, passwordTCL)

secondes_entre_2_requetes = 60 # secondes
duree_affichage = 5
nb_message_envoye = secondes_entre_2_requetes / duree_affichage
id_arret= 85 # Aubepin
nom_arret = "Aubepin"
ligne = "C26A" # C26 direction Grange blanche

# Fait la requete pour obtenir les données d'un arrêt et d'une ligne
# Renvoie un tab a 2 dim avec 1ere dim = nom du champ et 2eme valeur (un peu dico mais appelé ici DataFrame)
def GetDataTCL(auth, id_arret, ligne, type = "E"):
    param = f"id__eq={id_arret}&ligne__eq={ligne}&type__eq=E"
    param_csv="maxfeatures=-1&ds=.&separator=;&start=1"
    url = base_url + "?" + param + "&" + param_csv
    response = requests.get(url, auth=auth)
    text_csv = response.text
    return pd.read_csv(io.StringIO(text_csv), sep=";").sort_values("heurepassage")

# Créer message
def creer_message_delaipassage(data, nom_arret):
    liste_msg = []
    lenMaxLigne = 16
    # Creation de l'entete du message
    space = ' ' * (lenMaxLigne - len(data["ligne"][0]) - len("arret ") - len(nom_arret))
    titreLigne = data["ligne"][0] + space + "arret " + nom_arret + ";"

    for index, passage in data.iterrows():
        if index == 0:
            texte = "Prochain "
        elif index == 1:
            texte = "Suivant "
        else:
            texte = "Apres "

        # Date et heure spécifiée au format "AAAA-MM-JJ HH:MM:SS"
        date_passage_str = passage['heurepassage']
        # Conversion de la chaîne de caractères en objet datetime
        date_passage = datetime.strptime(date_passage_str, "%Y-%m-%d %H:%M:%S")
        # Obtenir l'heure actuelle
        now = datetime.now()
        # Calculer la différence de temps en secondes
        delta_seconds = (date_passage - now).total_seconds()
        # Convertir la différence de temps en minutes et donne un string
        delta_minutes_str = str(math.floor(delta_seconds / 60)) # Arrondi à l'inférieur
        
        space = ' ' * (lenMaxLigne - len(texte) - len(delta_minutes_str) - len(" min"))
        liste_msg.append(titreLigne + texte + space + delta_minutes_str + " min")
    return liste_msg


def creer_msg_queue(message : str, duree_affichage : int, temps_validite : int):
    now = datetime.now()
    date_peremption = now + timedelta(seconds = temps_validite)
    date_envoie_str = now.strftime('%Y-%m-%d %H:%M:%S')
    date_peremption_str = date_peremption.strftime('%Y-%m-%d %H:%M:%S')
    return f"TCL;{date_envoie_str};{date_peremption_str};{duree_affichage};{message}"

def main(duree_script: int):
    # Connexion queue
    credentials = pika.PlainCredentials(username="tcl", password="tcl")
    parameters = pika.ConnectionParameters(host=ip_rabbitMQ_service,
                                           port=port_rabbitMQ_service,
                                           virtual_host=virtual_host,
                                           credentials=credentials)
    connection = pika.BlockingConnection(parameters)
    channel = connection.channel()

    start_time_script = datetime.now()
    while(start_time_script + timedelta(minutes=duree_script) > datetime.now()):
        data = GetDataTCL(auth=authTCL, id_arret=id_arret, ligne=ligne)
        msg_delai = creer_message_delaipassage(data=data, nom_arret=nom_arret)
        for i in range(0, int(nb_message_envoye/len(msg_delai))):
            for m in msg_delai:
                msg_queue=creer_msg_queue(message=m, duree_affichage=duree_affichage, temps_validite=duree_affichage * 3)
                channel.basic_publish(exchange='',
                                      routing_key=queue_name,
                                      body=msg_queue)
                time.sleep(duree_affichage)
        # time.sleep(secondes_entre_2_requetes)

    channel.basic_publish(exchange='',
                        routing_key=queue_name,
                        body=creer_msg_queue(message="  Bye Bye  TCL  ;                ", 
                                             duree_affichage=10,
                                             temps_validite=120))
    connection.close()

if __name__ == '__main__':
    try:
        duree_script = 10 # en minutes

        if len(sys.argv) > 1:
            arg = sys.argv[1]
            if arg.isdigit():
                arg_int = int(arg)
                if arg_int > 0:
                    duree_script = arg_int

        print(f"Durée du script : {duree_script}  minute")
        main(duree_script)

    except KeyboardInterrupt:
        print('Interrupted')
        try:
            sys.exit(0)
        except SystemExit:
            os._exit(0)




Durée du script : 10  minute


: 

: 

## Lapin Viking steam

In [1]:
#!/usr/bin/env python
# Librairies
import pika, sys, os
import sys
import requests
from datetime import datetime, timedelta
import time
import requests
import steam.webauth as wa
import steam.webapi as api
import random

# Format du message : 
# nom_publisher
# date_envoie => %Y-%m-%d %H:%M:%S
# date_peremption => %Y-%m-%d %H:%M:%S.%3N"
# duree_affichage => en seconde
# ligne1_msg
# ligne2_msg
# Queue : format message 
MSG_TEST="Toto;2023-06-04 00:35:45;2024-06-04 00:35:45;5;Message ligne 1;Message ligne2"
FORMAT_MSG="nom_publisher;date_envoie(%Y-%m-%d %H:%M:%S);date_peremption(%Y-%m-%d %H:%M:%S);duree_affichage(secondes);ligne1_msg;ligne2_msg"

## Queue
# Queue parametre
ip_rabbitMQ_service="localhost"
port_rabbitMQ_service=5672
virtual_host="/"
queue_name="arduino"
usernameQueue=""
passwordQueue=""

# variables
id_jeu_valheim = 892970
api_key = ""
steam_id = ""

TEMPS_AFFICHAGE=5
TEMPS_VALIDITE=60

def recup_username(api_key, steam_id_user):
    username = "Not found"
    # URL de l'API Steam pour obtenir les informations de l'utilisateur
    url_player_info = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={api_key}&steamids={steam_id_user}"
    # Faites une requête GET à l'API Steam pour obtenir les informations de l'utilisateur
    response = requests.get(url_player_info)
    data = response.json()

    if data.get("response"):
        player_info = data["response"]["players"][0]
        player_name = player_info["personaname"]
        username = player_name
    return username

def recup_liste_amis(api_key, steam_id):
    liste_amis = {}
    # URL de l'API Steam pour obtenir la liste d'amis
    url_friends = f"http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?steamid={steam_id}&key={api_key}&relationship=friend"
    # Faites une requête GET à l'API Steam pour obtenir la liste d'amis
    response = requests.get(url_friends)
    data = response.json()

    if data.get("friendslist"):
        friends = data["friendslist"]["friends"]
        for friend in friends:
            # Obtenir l'ID Steam de l'ami
            friend_id = friend["steamid"]
            username = recup_username(api_key, friend_id)
            liste_amis[friend_id] = username
    return liste_amis

def copains_sur_un_jeu(api_key, liste_amis, id_jeu):
    nbAmis = 0
    for id_ami in liste_amis.keys():
        # URL de l'API Steam pour obtenir les infos sur l'ami
        url_info_ami = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={api_key}&steamids={id_ami}"
        # Faites une requête GET à l'API Steam pour obtenir les jeux possédés par l'ami
        response = requests.get(url_info_ami)
        info_ami = response.json()
        if info_ami.get("response"):
            # print(info_ami)
            if info_ami["response"]["players"][0].get("gameid"):
                if info_ami["response"]["players"][0]["gameid"] == str(id_jeu):
                    print(liste_amis[id_ami] + " est sur le jeu " + str(id_jeu))
                    nbAmis +=1
    return nbAmis

def creer_msg_joueur_sur_jeu(nb_joueur : int, nom_jeu : str):
    ligne1      = ["    VALHEIM     ", "   Par Odin !   ", "   Par Thor !   ", "Drakkar en mer !", "  Skal a toi !  ", "Fureur viking ! ", "Valhalla attend "]
    ligne2_sing = [" brave Viking  " , " fier guerrier "  , " fier guerrier "  , " copain en jeu " , " brave Nordique"]
    ligne2_plu = [" braves Vikings "  , " fiers guerriers" , " fiers guerriers" , " copains en jeu", " viking en jeu "]

    msg = ligne1[random.randint(0, len(ligne1))] + ";" 
    if(nb_joueur > 1):
        msg += ligne2_plu[random.randint(0, len(ligne2_plu))]
    else:
        msg += ligne2_sing[random.randint(0, len(ligne2_sing))]
    return msg 

def creer_msg_queue(message : str, duree_affichage: int, temps_validite: int):
    now = datetime.now()
    date_peremption = now + timedelta(seconds = temps_validite)
    date_envoie_str = now.strftime('%Y-%m-%d %H:%M:%S')
    date_peremption_str = date_peremption.strftime('%Y-%m-%d %H:%M:%S')
    return f"TCL;{date_envoie_str};{date_peremption_str};{duree_affichage};{message}"


def main(duree_script: int):
    # Connexion queue
    credentials = pika.PlainCredentials(username=usernameQueue, password=passwordQueue)
    parameters = pika.ConnectionParameters(host=ip_rabbitMQ_service,
                                           port=port_rabbitMQ_service,
                                           virtual_host=virtual_host,
                                           credentials=credentials)
    connection = pika.BlockingConnection(parameters)
    channel = connection.channel()

    liste_amis = recup_liste_amis(api_key=api_key, steam_id=steam_id)

    start_time_script = datetime.now()
    while(start_time_script + timedelta(minutes=duree_script) > datetime.now()):
        nb_copains_en_jeu = copains_sur_un_jeu(api_key=api_key, liste_amis=liste_amis, id_jeu=id_jeu_valheim)
        if(nb_copains_en_jeu > 0):
            msg = creer_msg_joueur_sur_jeu(nb_joueur=nb_copains_en_jeu, nom_jeu='Valheim')
            msg_queue = creer_msg_queue(msg, TEMPS_AFFICHAGE, TEMPS_VALIDITE)
            channel.basic_publish(exchange='',
                                routing_key=queue_name,
                                body=creer_msg_queue(message=" Bye Bye steam  ; Viking go dodo ", 
                                                    duree_affichage=10,
                                                    temps_validite=120))
            time.sleep(TEMPS_AFFICHAGE * 2)
        time.sleep(TEMPS_VALIDITE * 2)
   
    connection.close()

if __name__ == '__main__':
    try:
        duree_script = 60 # en minutes

        if len(sys.argv) > 1:
            arg = sys.argv[1]
            if arg.isdigit():
                arg_int = int(arg)
                if arg_int > 0:
                    duree_script = arg_int

        print(f"Durée du script : {duree_script}  minute")
        main(duree_script)

    except KeyboardInterrupt:
        print('Interrupted')
        try:
            sys.exit(0)
        except SystemExit:
            os._exit(0)


Durée du script : 60  minute
